In [11]:
import pandas as pd
import datetime as dt
from datetime import datetime
import warnings
import dask.dataframe as dd 
import os 
warnings.filterwarnings("ignore")

In [12]:
os.getcwd()

'/Users/edouardpenel/Desktop/ENSAE 2A/Statapp/Données/Partagé/Projet-Statapp/Lisa'

In [13]:
path = os.path.dirname(os.getcwd())
print(path)
path2 = os.path.dirname(os.path.dirname(os.path.dirname(path)))
print(path2)


/Users/edouardpenel/Desktop/ENSAE 2A/Statapp/Données/Partagé/Projet-Statapp
/Users/edouardpenel/Desktop/ENSAE 2A/Statapp


In [14]:
#! pip install dask

In [19]:
df_cct_dask = dd.read_csv("/Users/edouardpenel/Desktop/ENSAE 2A/Statapp/Code/Données/credit_card_transactions-ibm_v2.csv")

In [3]:
df_cct = pd.read_csv("D:\Statapp\data\credit_card_transactions-ibm_v2.csv")
df_cards = pd.read_csv("D:\Statapp\data\sd254_cards.csv")
df_users = pd.read_csv("D:\Statapp\data\sd254_users.csv")
df_cctu0 = pd.read_csv(r"D:\Statapp\data\User0_credit_card_transactions.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Statapp\\data\\credit_card_transactions-ibm_v2.csv'

In [4]:
#dropping $ sign in "Amount"
df_cct["Amount"] = df_cct["Amount"].str.replace("$","")

## Par utilisateur et par carte

In [5]:
df_test = df_cct[(df_cct['User'].isin([0,1,2,3,4,5]))]
df_test[['Hours', 'Minutes']] = df_test.Time.str.split(":", expand=True)
df_test["Hours"] = df_test["Hours"].astype(int)
df_test["Minutes"] = df_test["Minutes"].astype(int)
df_test["Amount"] = df_test["Amount"].astype(float)

df_test["Date"] = pd.to_datetime(df_test[["Year",'Month','Day']], format = '%Y:%M:%D').values

b = pd.to_datetime(df_test[["Year",'Month','Day','Hours','Minutes']], format = '%Y:%M:%D%:%H:%M')
df_test["Timestamp"] = b.values
df_test['delta_t_card'] = (b-b.shift())

df_test = df_test[['User', 'Card', 'Date', 'Timestamp','Amount', 'Use Chip',
       'Merchant Name', 'Merchant City', 'Merchant State', 'Zip', 'MCC',
       'Errors?', 'Is Fraud?', 'delta_t_card']]

df_test.loc[~df_test.duplicated("User", keep='first'), "delta_t_card"] = 0
df_test.loc[~df_test.duplicated("Card", keep='first'), "delta_t_card"] = 0
df_test["Amount"] = df_test["Amount"].astype(float)
a1 = df_test.groupby(["User","Card", "Date"])["Amount"].sum().values
a2 = df_test.groupby(["User","Card", "Date"])["Amount"].sum().index
nb1 = df_test.groupby(["User","Card", "Date"])["Amount"].size().values
nb2 = df_test.groupby(["User","Card", "Date"])["Amount"].size().index
df1 = pd.DataFrame(a1,a2,columns=["Amount/day"]).reset_index()
df2 = pd.DataFrame(nb1,nb2,columns=["nb_trans/day"]).reset_index()
df1["nb_trans/day"] = df2["nb_trans/day"]
df_test = pd.merge(df_test,df1,on = ["User","Card","Date"])
df_test

,User,Card,Date,Timestamp,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,delta_t_card,Amount/day,nb_trans/day
0,0,0,2002-09-01,2002-09-01 06:21:00,134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,NaN,No,0,172.57,2
1,0,0,2002-09-01,2002-09-01 06:42:00,38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No,0 days 00:21:00,172.57,2
2,0,0,2002-09-02,2002-09-02 06:22:00,120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No,0 days 23:40:00,249.29,2
3,0,0,2002-09-02,2002-09-02 17:45:00,128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,NaN,No,0 days 11:23:00,249.29,2
4,0,0,2002-09-03,2002-09-03 06:23:00,104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,NaN,No,0 days 12:38:00,190.90,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121202,5,2,2020-02-27,2020-02-27 16:10:00,51.46,Chip Transaction,3635551857898739641,Davenport,IA,52804.0,5310,NaN,No,0 days 03:50:00,200.11,4
121203,5,2,2020-02-27,2020-02-27 16:38:00,60.00,Chip Transaction,-4282466774399734331,Cedarburg,WI,53012.0,4829,NaN,No,0 days 00:28:00,200.11,4
121204,5,2,2020-02-27,2020-02-27 21:39:00,17.08,Chip Transaction,6657473015144657322,Davenport,IA,52803.0,5813,NaN,No,0 days 05:01:00,200.11,4
121205,5,2,2020-02-28,2020-02-28 16:44:00,83.64,Swipe Transaction,1913477460590765860,East Troy,WI,53120.0,5300,NaN,No,0 days 19:05:00,197.36,2


## Par utilisateur

In [68]:
df_test2 = df_cct[df_cct["User"].isin([0,1,2,3,4,5])] # 6 utilisateurs

#Creating timestamp
df_test2[['Hours', 'Minutes']] = df_test2.Time.str.split(":", expand=True)
df_test2["Hours"] = df_test2["Hours"].astype(int)
df_test2["Minutes"] = df_test2["Minutes"].astype(int)
df_test2["Amount"] = df_test2["Amount"].astype(float)

df_test2 = df_test2.sort_values(["User", "Year","Month","Day","Hours","Minutes"], ascending=[True,True,True,True,True,True])

df_test2["Date"] = pd.to_datetime(df_test2[["Year",'Month','Day']], format = '%Y:%M:%D').values
a = pd.to_datetime(df_test2[["Year",'Month','Day','Hours','Minutes']], format = '%Y:%M:%D%:%H:%M')
df_test2["Timestamp"] = a.values
df_test2['delta_t'] = (a-a.shift())

#Sorting by user and timestamp
df_test2 = df_test2[['User', 'Card', 'Date', 'Timestamp','Amount', 'Use Chip',
       'Merchant Name', 'Merchant City', 'Merchant State', 'Zip', 'MCC',
       'Errors?', 'Is Fraud?', 'delta_t']]

#Setting time to 0 for each new user
df_test2.loc[~df_test2.duplicated("User", keep='first'), "delta_t"] = 0
df_test2 = pd.merge(df_test2, df_test[["User","Card","Date","Timestamp","Amount","delta_t_card"]], how='left')


In [69]:
#Determining the daily amount of transactions
day_amt_v = df_test2.groupby(["User", "Date"])["Amount"].sum().values
day_amt_i = df_test2.groupby(["User", "Date"])["Amount"].sum().index
#Determining the daily number of transactions
day_nbt_v = df_test2.groupby(["User", "Date"])["Amount"].size().values
day_nbt_i = df_test2.groupby(["User", "Date"])["Amount"].size().index

df1 = pd.DataFrame(day_amt_v,day_amt_i,columns=["day_amt"]).reset_index()
df2 = pd.DataFrame(day_nbt_v,day_nbt_i,columns=["day_nbt"]).reset_index()


df1["day_nbt"] = df2["day_nbt"]
df1["day_av_nbt"] = df1["day_nbt"].expanding().mean()
df1["Ratio 1"] = df1["day_nbt"]/df1["day_av_nbt"] # ratio 1 : determining the share of day i nb of transaction
# out of the daily average nb of transactions until day i


df_test2 = pd.merge(df_test2,df1,on = ["User","Date"]).reset_index(drop=True)
df_test2

,User,Card,Date,Timestamp,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,delta_t,delta_t_card,day_amt,day_nbt,day_av_nbt,Ratio 1
0,0,0,2002-09-01,2002-09-01 06:21:00,134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,NaN,No,0,0,172.57,2,2.000000,1.000000
1,0,0,2002-09-01,2002-09-01 06:42:00,38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No,0 days 00:21:00,0 days 00:21:00,172.57,2,2.000000,1.000000
2,0,0,2002-09-02,2002-09-02 06:22:00,120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No,0 days 23:40:00,0 days 23:40:00,249.29,2,2.000000,1.000000
3,0,0,2002-09-02,2002-09-02 17:45:00,128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,NaN,No,0 days 11:23:00,0 days 11:23:00,249.29,2,2.000000,1.000000
4,0,0,2002-09-03,2002-09-03 06:23:00,104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,NaN,No,0 days 12:38:00,0 days 12:38:00,190.90,2,2.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121202,5,2,2020-02-27,2020-02-27 21:39:00,17.08,Chip Transaction,6657473015144657322,Davenport,IA,52803.0,5813,NaN,No,0 days 03:42:00,0 days 05:01:00,290.36,5,3.493989,1.431029
121203,5,0,2020-02-28,2020-02-28 16:38:00,60.00,Chip Transaction,-4282466774399734331,Cedarburg,WI,53012.0,4829,NaN,No,0 days 18:59:00,0 days 22:41:00,334.97,4,3.494004,1.144818
121204,5,2,2020-02-28,2020-02-28 16:44:00,83.64,Swipe Transaction,1913477460590765860,East Troy,WI,53120.0,5300,NaN,No,0 days 00:06:00,0 days 19:05:00,334.97,4,3.494004,1.144818
121205,5,0,2020-02-28,2020-02-28 17:47:00,77.61,Chip Transaction,4722913068560264812,Davenport,IA,52803.0,5411,NaN,No,0 days 01:03:00,0 days 01:09:00,334.97,4,3.494004,1.144818


In [70]:
df6 = pd.DataFrame()
#Determining the share of week i number of transactions out of the weekly average number of transactions
#until week i : ratio 2

for i in range(len(df_test2["User"].unique())):
    df4 = pd.DataFrame(df_test2[df_test2["User"] == i][['User',"Date"]])
    wk_nbt_i = df_test2[df_test2["User"] == i].groupby([pd.Grouper(key="Date", freq="7d")]).size().index
    wk_nbt_v = df_test2[df_test2["User"] == i].groupby([pd.Grouper(key="Date", freq="7d")]).size().values

#Determining the share of week i transactions amount out of the weekly average transactions amount
#until week i : ratio 3
    wk_amt_i = df_test2[df_test2["User"] == i].groupby([pd.Grouper(key="Date", freq="7d")])["Amount"].sum().index
    wk_amt_v = df_test2[df_test2["User"] == i].groupby([pd.Grouper(key="Date", freq="7d")])["Amount"].sum().values
    df3 = pd.DataFrame(wk_nbt_v,wk_nbt_i, columns=["wk_i_nbt"]).reset_index()
    df3["User"] = i
    df3["wk_av_nbt"] = df3["wk_i_nbt"].expanding().mean()
    df3["Ratio 2"] = df3["wk_i_nbt"]/df3["wk_av_nbt"] 
    df3["wk_i_amt"] = wk_amt_v
    df3["wk_av_amt"] = df3["wk_i_amt"].expanding().mean()
    df3["Ratio 3"] = df3["wk_i_amt"]/df3["wk_av_amt"] #
    
    df5 = pd.merge(df4,df3,how='left',on=["User","Date"])
    df6 = pd.concat([df6,df5], axis=0)
    df6[['wk_i_nbt', 'wk_av_nbt', 'Ratio 2', 'wk_i_amt',
       'wk_av_amt', 'Ratio 3']] = df6[['wk_i_nbt', 'wk_av_nbt', 'Ratio 2', 'wk_i_amt',
       'wk_av_amt', 'Ratio 3']].fillna(method="ffill")
    

df6.reset_index(drop=True, inplace=True)

In [93]:
df_test3 = pd.concat([df_test2,df6[['wk_i_nbt', 'wk_av_nbt', 'Ratio 2', 'wk_i_amt',
       'wk_av_amt', 'Ratio 3']]], axis=1)


In [96]:
df_test3[['User', 'Card', 'Date', 'Timestamp', 'Amount', 'Use Chip',
       'Is Fraud?', 'delta_t', 'delta_t_card', 'day_amt', 'day_nbt',
       'day_av_nbt', 'Ratio 1', 'wk_i_nbt', 'wk_av_nbt', 'Ratio 2', 'wk_i_amt',
       'wk_av_amt', 'Ratio 3']]

,User,Card,Date,Timestamp,Amount,Use Chip,Is Fraud?,delta_t,delta_t_card,day_amt,day_nbt,day_av_nbt,Ratio 1,wk_i_nbt,wk_av_nbt,Ratio 2,wk_i_amt,wk_av_amt,Ratio 3
0,0,0,2002-09-01,2002-09-01 06:21:00,134.09,Swipe Transaction,No,0,0,172.57,2,2.000000,1.000000,16.0,16.000000,1.000000,1380.96,1380.960000,1.000000
1,0,0,2002-09-01,2002-09-01 06:42:00,38.48,Swipe Transaction,No,0 days 00:21:00,0 days 00:21:00,172.57,2,2.000000,1.000000,16.0,16.000000,1.000000,1380.96,1380.960000,1.000000
2,0,0,2002-09-02,2002-09-02 06:22:00,120.34,Swipe Transaction,No,0 days 23:40:00,0 days 23:40:00,249.29,2,2.000000,1.000000,16.0,16.000000,1.000000,1380.96,1380.960000,1.000000
3,0,0,2002-09-02,2002-09-02 17:45:00,128.95,Swipe Transaction,No,0 days 11:23:00,0 days 11:23:00,249.29,2,2.000000,1.000000,16.0,16.000000,1.000000,1380.96,1380.960000,1.000000
4,0,0,2002-09-03,2002-09-03 06:23:00,104.71,Swipe Transaction,No,0 days 12:38:00,0 days 12:38:00,190.90,2,2.000000,1.000000,16.0,16.000000,1.000000,1380.96,1380.960000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121202,5,2,2020-02-27,2020-02-27 21:39:00,17.08,Chip Transaction,No,0 days 03:42:00,0 days 05:01:00,290.36,5,3.493989,1.431029,16.0,22.877637,0.699373,947.95,1342.215295,0.706258
121203,5,0,2020-02-28,2020-02-28 16:38:00,60.00,Chip Transaction,No,0 days 18:59:00,0 days 22:41:00,334.97,4,3.494004,1.144818,16.0,22.877637,0.699373,947.95,1342.215295,0.706258
121204,5,2,2020-02-28,2020-02-28 16:44:00,83.64,Swipe Transaction,No,0 days 00:06:00,0 days 19:05:00,334.97,4,3.494004,1.144818,16.0,22.877637,0.699373,947.95,1342.215295,0.706258
121205,5,0,2020-02-28,2020-02-28 17:47:00,77.61,Chip Transaction,No,0 days 01:03:00,0 days 01:09:00,334.97,4,3.494004,1.144818,16.0,22.877637,0.699373,947.95,1342.215295,0.706258
